In [66]:
import json
import numpy as np
import matplotlib.pyplot as plt
from tiatoolbox.tools.patchextraction import get_patch_extractor
import os
import skimage
from tiatoolbox.wsicore.wsireader import VirtualWSIReader
import cv2
from tqdm.auto import tqdm

In [ ]:
splits_file_path = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/splits.json"
with open(splits_file_path, "r") as file:
    splits = json.load(file)

In [ ]:
save_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/dilation/patches_new"
tile_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/dilation/tiles"

In [ ]:
def img_to_shape(a, shape):
    y_, x_ = shape
    y, x = a.shape[0], a.shape[1]
    y_pad = y_ - y
    x_pad = x_ - x
    return np.pad(
        a,
        (
            (y_pad // 2, y_pad // 2 + y_pad % 2),
            (x_pad // 2, x_pad // 2 + x_pad % 2),
            (0, 0),
        ),
        mode="constant",
    )


def mask_to_shape(a, shape):
    y_, x_ = shape
    y, x = a.shape[0], a.shape[1]
    y_pad = y_ - y
    x_pad = x_ - x
    return np.pad(
        a,
        ((y_pad // 2, y_pad // 2 + y_pad % 2), (x_pad // 2, x_pad // 2 + x_pad % 2)),
        mode="constant",
    )


def get_cell_centers(cell_mask):
    mask_label = skimage.measure.label(cell_mask)
    stats = skimage.measure.regionprops(mask_label)
    xs = []
    ys = []
    for region in stats:
        centroid = region["centroid"]
        xs.append(centroid[1])
        ys.append(centroid[0])
    return xs, ys


def extract_cell_img_mask(tile_img, tile_mask, img_name):
    img_reader = VirtualWSIReader.open(tile_img)
    mask_reader = VirtualWSIReader.open(tile_mask, mode="bool")

    extractor = get_patch_extractor(
        input_img=img_reader,
        method_name="slidingwindow",
        patch_size=(128, 128),
        resolution=0,
        units="level",
        # min_mask_ratio=0.3
    )

    results = {}
    idx = 0

    for coords in extractor.coordinate_list:
        img_patch = img_reader.read_bounds(coords, resolution=0, units="level")
        mask_patch = mask_reader.read_bounds(coords, resolution=0, units="level")

        img_mask_file = np.zeros(shape=(128, 128, 4), dtype=np.uint8)
        img_mask_file[:, :, 0:3] = img_patch
        img_mask_file[:, :, 3] = mask_patch[:, :, 0]

        non_zeros = np.count_nonzero(mask_patch[:, :, 0])

        img_name_without_ext = os.path.splitext(img_name)[0]

        if non_zeros > 0:
            save_name = f"{img_name_without_ext}_{idx}_p.npy"
        else:
            save_name = f"{img_name_without_ext}_{idx}_n.npy"

        results[save_name] = img_mask_file

        idx += 1

    return results


def erode_cell_mask(mask):
    "Reduce mask size of a single cell from 49 to 29 px"
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.erode(mask, kernel, iterations=1)
    return mask

In [71]:
fold = 5
fold_save_dir = os.path.join(save_dir, f"fold_{fold}")
os.makedirs(fold_save_dir, exist_ok=True)

fold_tile_dir = os.path.join(tile_dir, f"fold_{fold}")
fold_files = os.listdir(fold_tile_dir)


for tile_file in tqdm(fold_files):
    print(tile_file)
    tile_path = os.path.join(fold_tile_dir, tile_file)
    tile = np.load(tile_path)
    print(tile.shape, tile.dtype)

    tile_img = tile[:, :, 0:3]
    mask = tile[:, :, 3]
    # mask = erode_cell_mask(mask)

    if tile_img.shape[0] < 128 and tile_img.shape[1] < 128:
        tile_img = img_to_shape(tile_img, (128, 128))
        mask = mask_to_shape(mask, (128, 128))

    # xs, ys = get_cell_centers(mask)
    # fig, axes = plt.subplots(1,2)
    # axes[0].imshow(tile_img)
    # axes[0].scatter(xs, ys, s=3, alpha=0.5, color='red')
    # axes[1].imshow(mask)
    # plt.show()

    patch_result = extract_cell_img_mask(tile_img, mask, tile_file)

    for result in patch_result.items():
        fn = result[0]
        patch = result[1]

        save_path = os.path.join(fold_save_dir, fn)
        np.save(save_path, patch)
        # print(fn)
        # fig, axes = plt.subplots(1,2)
        # rgb = patch[:,:,0:3]
        # mask = patch[:,:,3]
        # axes[0].imshow(rgb, alpha=1)
        # axes[0].imshow(mask, cmap='gray', alpha=0.4)
        # axes[1].imshow(mask)
        # plt.show()

  0%|          | 0/49 [00:00<?, ?it/s]

|2024-06-14|16:29:30.349| [WARNING] Raw data is None.
|2024-06-14|16:29:30.349| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.351| [WARNING] Raw data is None.
|2024-06-14|16:29:30.352| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.504| [WARNING] Raw data is None.
|2024-06-14|16:29:30.504| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.506| [WARNING] Raw data is None.
|2024-06-14|16:29:30.506| [WARNING] Unknown scale (no objective_power or mpp)


136B_[55424, 17753, 56685, 18932].npy
(1179, 1261, 5) uint8
136B_[64379, 16227, 65677, 17403].npy
(1176, 1298, 5) uint8


|2024-06-14|16:29:30.666| [WARNING] Raw data is None.
|2024-06-14|16:29:30.667| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.669| [WARNING] Raw data is None.
|2024-06-14|16:29:30.669| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.821| [WARNING] Raw data is None.
|2024-06-14|16:29:30.821| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:30.824| [WARNING] Raw data is None.
|2024-06-14|16:29:30.824| [WARNING] Unknown scale (no objective_power or mpp)


136B_[73791, 8482, 75066, 9666].npy
(1184, 1275, 5) uint8
198B_[19017, 3481, 20279, 4695].npy
(1214, 1262, 5) uint8


...

TC_S01_P000147_C0001_B104_[44555, 80802, 45637, 81863].npy
(1061, 1082, 5) uint8
TC_S01_P000158_C0001_B101_[102793, 79018, 104054, 80142].npy
(1124, 1261, 5) uint8


|2024-06-14|16:29:34.427| [WARNING] Raw data is None.
|2024-06-14|16:29:34.428| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:34.430| [WARNING] Raw data is None.
|2024-06-14|16:29:34.431| [WARNING] Unknown scale (no objective_power or mpp)


TC_S01_P000158_C0001_B101_[103602, 82756, 104838, 83891].npy
(1135, 1236, 5) uint8


|2024-06-14|16:29:34.634| [WARNING] Raw data is None.
|2024-06-14|16:29:34.634| [WARNING] Unknown scale (no objective_power or mpp)
|2024-06-14|16:29:34.636| [WARNING] Raw data is None.
|2024-06-14|16:29:34.636| [WARNING] Unknown scale (no objective_power or mpp)


TC_S01_P000158_C0001_B101_[114472, 87563, 115698, 88757].npy
(1194, 1226, 5) uint8
